# Evaluation
Ce notebook sert à importer un modèle entrainé dans le but de l'évaluer. 
Actuellement il sert juste à visualiser mais le but était d'entrainer un modèle sur plusieurs données d'entrée puis de l'évaluer sur une nouvelle donnée autre que celle servie pour l'entrainement. En effet, lorsqu'on importe un modèle entrainé pour tester sur de nouvelles valeurs sur le même intervalle d'études, l'entrainement est plus rapide. C'est logique car les poids et les biais importés du modèle entrainé sont plus proches que ceux généré aléatoires lors de l'initialisation du modèle.

In [1]:
######################################
# Pour le fonctionnement du notebook #
######################################
import numpy as np
from IPython.display import display, clear_output, HTML
import torch
import torch.nn as nn
from torchsummary import summary
%autosave 300

##################################################
# Pour déterminer si on utilise un cpu ou un gpu #
##################################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("On utilise ",device)
print(torch.cuda.is_available())
####################
# Pour l'affichage #
####################
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
def animate_show(anim):
    html_version = HTML(anim.to_jshtml())
    return html_version
import matplotlib.animation as animation

Autosaving every 300 seconds
On utilise  cuda:0
True


In [2]:
#####################################
# Choix de la fonction d'activation #
#####################################
# activation = nn.Sin();
activation = nn.Tanh()
# activation = nn.Sigmoid()
# activation = nn.Tanhshrink()
# activation = nn.Hardtanh(-2, 2)
# activation = nn.Hardswish()
# activation = nn.SiLU()

########################################################
# Création d'une classe qui crée le réseau de neurones #
########################################################
class Reseau(nn.Module):
    def __init__(self,couches):
        # L=[2,3,3,3,1];
        super().__init__()
        # Création de la couche d'entrée
        self.couche_entree = nn.Linear(couches[0],couches[1])
        # Création des couches cachées
        self.couches_cachees = nn.ModuleList()
        for i in range(1, len(couches)-2):
            self.couches_cachees.append(nn.Linear(couches[i], couches[i+1]))
        # Création de la couche de sortie
        self.couches_sortie = nn.Linear(couches[-2], couches[-1])
    
    # Propagation
    def forward(self,data):
        u = activation(self.couche_entree(data))
        for l in self.couches_cachees:
            u = activation(l(u))
        u = self.couches_sortie(u)
        return u
##################################
# Création du réseau de neurones #
##################################
# Création de notre réseau
L = [2,3,3,3,1]
modele = Reseau(L).to(device)
# display(modele) 
# summary(modele,(1,2))
# modele.load_state_dict(torch.load("./test_couches/modeles/modele_cc_n4.pt"))
modele.eval()

Reseau(
  (couche_entree): Linear(in_features=2, out_features=3, bias=True)
  (couches_cachees): ModuleList(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): Linear(in_features=3, out_features=3, bias=True)
  )
  (couches_sortie): Linear(in_features=3, out_features=1, bias=True)
)

In [3]:
######################
# Paramètres Burgers #
######################
# Viscosité cinématique
nu = torch.tensor(0.01/np.pi)
# Discrétisation spatiale
xa = 0; xb = 2; pasX = 0.04
# Discrétisation temporelle
t0 = 0; tf = 1; pasT = 0.02
# Donnée initiale
Ud = 0; Ug = 1; x0 = 0.5
summary(modele,(1,2))
print(modele.couches_cachees[0].weight)

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 3]                9
├─ModuleList: 1                          []                        --
|    └─Linear: 2-1                       [-1, 1, 3]                12
|    └─Linear: 2-2                       [-1, 1, 3]                12
├─Linear: 1-2                            [-1, 1, 1]                4
Total params: 37
Trainable params: 37
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
Parameter containing:
tensor([[ 0.0668,  0.0119,  0.3610],
        [ 0.4286,  0.0511, -0.3447],
        [-0.2344,  0.2902, -0.4580]], device='cuda:0', requires_grad=True)


In [4]:
modele = torch.load('./modele_EP_5000BIS.pt').to(device)
print(modele.couches_cachees[0].weight)
modele.eval()

Parameter containing:
tensor([[-1.9426,  2.0738,  1.6041],
        [ 1.6896, -0.9522, -1.5468],
        [ 3.2136, -2.3863, -1.9392]], device='cuda:0', requires_grad=True)


Reseau(
  (couche_entree): Linear(in_features=2, out_features=3, bias=True)
  (couches_cachees): ModuleList(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): Linear(in_features=3, out_features=3, bias=True)
  )
  (couches_sortie): Linear(in_features=3, out_features=1, bias=True)
)

In [5]:
############################################
# Solution exacte de l'équation de Burgers #
############################################
def sol_exacte_Burgers(x,t,Ud,Ug,x0):
    u_exact=np.zeros((len(t),len(x)))
    for j in range(len(t)):
        if t[j]==0:
            for i in range(len(x)):
                if x[i]<=x0:
                    u_exact[j,i]=Ug
                else:
                    u_exact[j,i]=Ud
        else:
            for i in range(len(x)):
                if Ud<Ug:
                    sigma=0.5*(Ud**2-Ug**2)/(Ud-Ug)
                    if (x[i]-x0)/t[j]<sigma:
                        u_exact[j,i]=Ug
                    else:
                        u_exact[j,i]=Ud
                else:
                    if (x[i]-x0)/t[j]<Ug:
                        u_exact[j,i]=Ug
                    elif Ug<=(x[i]-x0)/t[j]<=Ud:
                        u_exact[j,i]=(x[i]-x0)/t[j]
                    else:
                        u_exact[j,i]=Ud
    return u_exact

# Calcul de la solution exacte de l'équation de Burgers
y_sol_exacte=sol_exacte_Burgers(np.arange(xa,xb,pasX),np.arange(t0,tf,pasT),Ud,Ug,x0)

In [6]:
#################################################
# Projection 3D des résultats de l'entrainement #
#################################################
# Création de la figure
fig2 = plt.figure()
ax = plt.subplot(projection='3d')
# Création des données de la figure à partir du modèle PINN entrainé
x=np.arange(xa,xb,pasX)
t=np.arange(t0,tf,pasT)
print(len(x),len(t))
ms_x, ms_t = np.meshgrid(x, t)
x = np.ravel(ms_x).reshape(-1,1)
t = np.ravel(ms_t).reshape(-1,1)
pt_x = torch.from_numpy(x).float().to(device)
pt_t = torch.from_numpy(t).float().to(device)
# Evaluation
pt_u = modele(torch.cat((pt_x, pt_t),1).to(device))
u=pt_u.data.cpu().numpy()
ms_u = u.reshape(ms_x.shape)
# Affichage des résultats
surf = ax.plot_surface(ms_x,ms_t,ms_u, cmap=cm.coolwarm,edgecolor='none')
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
fig2.colorbar(surf, shrink=0.4, aspect=5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

50 50


In [7]:
#######################################
# Animation 1D à partir des résultats #
#######################################
# Création de la figure pour l'affichage de l'animation
fig3 = plt.figure() 
plt.xlabel("Position x")
plt.ylabel("Burgers u")
plt.title("Evolution de la solution de Burgers au cours du temps")
# Pour l'affichage des courbes
line1, = plt.plot([], [], label="Solution PINN") 
line2, = plt.plot([], [], label="Solution exacte")
plt.legend()
# Pour l'affichage du temps
time_template = 'time = %.1fs'
time_text = plt.text(0.0, xb+0.1, '')
# Pour la taille du graphique
marge=0.25
plt.xlim(xa-marge, xb+marge)
plt.ylim(-marge, max(Ud,Ug)+marge)
# Création de l'animation
def animate(i):
    y = ms_u[i,:]
    x = ms_x[i,:]
    line1.set_data(x, y)
    line2.set_data(x, y_sol_exacte[i,:])
    time_text.set_text(time_template % (i*pasT))
    return line1, line2, time_text
ani = animation.FuncAnimation(fig3, animate, frames=len(ms_u), blit=True, interval=1, repeat=False)
plt.close(fig3)
animate_show(ani)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …